# 第四次作业

In [1]:
import numpy as np
# 棋盘类
class Env(object):
    def __init__(self, row, column):
        self.rewards = np.full((row, column), -1.0)
        self.states = np.ones((row, column), dtype=np.int)  # 若有障碍物可设置状态为-1

        self.index_list = [index for index, x in np.ndenumerate(self.states)]  # 可到达位置的索引号
        self.ns_table = dict()  # 下一步可能移动到的位置
        self._init_next_state_table()

        self.rewards[0, 0] = 0
        self.rewards[row - 1, column - 1] = 0

        self.terminal = [(0, 0), (row - 1, column - 1)]

    # 每个位置下一步可能到达位置的初始化
    def _init_next_state_table(self):
        for i, j in self.index_list:
            next_states = list()
            if (i - 1, j) in self.index_list:
                next_states.append((i - 1, j))
            else:
                next_states.append((i, j))
            if (i + 1, j) in self.index_list:
                next_states.append((i + 1, j))
            else:
                next_states.append((i, j))
            if (i, j - 1) in self.index_list:
                next_states.append((i, j - 1))
            else:
                next_states.append((i, j))
            if (i, j + 1) in self.index_list:
                next_states.append((i, j + 1))
            else:
                next_states.append((i, j))
            self.ns_table[(i, j)] = next_states

    def get_reward(self, i, j):
        return self.rewards[i, j]

    def get_states(self):
        return self.states

In [2]:
#  物体类
class Robot(object):
    def __init__(self, env, gamma):
        self._env = env
        self._gamma = gamma  # 衰减因子
        self.values = np.zeros_like(env.get_states(), dtype=np.float32)

    def best_value_func(self, i, j):
        if (i, j) in self._env.terminal:
            return self._env.get_reward(i, j)
        else:
            return self._env.get_reward(i, j) + self._gamma * np.sum(self.next_states_expected_value(i, j))

    def update_values(self):
        next_values = np.zeros_like(self.values, dtype=np.float32)
        for i, j in self._env.index_list:
            next_values[i, j] = self.best_value_func(i, j)
        self.values = next_values

    def next_states_expected_value(self, i, j):
        next_values = []
        ps = [0.25, 0.25, 0.25, 0.25]
        values = [self.values[i, j] for i, j in self._env.ns_table[(i, j)]]
        next_values.append(np.multiply(values, ps))
        return next_values

    #  动作决策
    def best_policy(self, i, j):
        if (i, j) not in self._env.terminal:
            print("(%d, %d)" % (i, j), end=" -> ")
            next_states = self._env.ns_table[(i, j)]
            best_state_index = np.argmax(self.next_states_expected_value(i, j))
            best_state = next_states[best_state_index]
            return self.best_policy(best_state[0], best_state[1])
        print("(%d, %d)" % (i, j))
        return


In [3]:
gamma = 1
env = Env(4, 4)
robot = Robot(env, gamma)

for _ in range(200):
    robot.update_values()
print(robot.values)

[[  0.       -13.999758 -19.999641 -21.999598]
 [-13.999758 -17.999683 -19.999641 -19.999641]
 [-19.999641 -19.999641 -17.999683 -13.999758]
 [-21.999598 -19.999641 -13.999758   0.      ]]


In [4]:
robot.best_policy(1, 2)

(1, 2) -> (2, 2) -> (3, 2) -> (3, 3)
